In [1]:
from docling.document_converter import DocumentConverter

/home/athekunal/MCP-SEC-Filings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "ds4sd/SmolDocling-256M-preview"
source = "/home/athekunal/MCP-SEC-Filings/SEC_EDGAR_FILINGS/AMZN-2024/amzn-20240331-10-Q1.pdf"  # document per local path or URL

from pdf2image import convert_from_path, convert_from_bytes
images = convert_from_path(source)

In [8]:
import pathlib 

pathlib.Path(source).stem

'amzn-20240331-10-Q1'

In [7]:
type(images[0])

PIL.PpmImagePlugin.PpmImageFile

In [4]:
# Prerequisites:
# pip install torch
# pip install docling_core
# pip install transformers

import torch
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
from pathlib import Path

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize processor and model
processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
model = AutoModelForVision2Seq.from_pretrained(
    "ds4sd/SmolDocling-256M-preview",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Convert this page to docling."}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[images[3]], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=8192)
prompt_length = inputs.input_ids.shape[1]
trimmed_generated_ids = generated_ids[:, prompt_length:]
doctags = processor.batch_decode(
    trimmed_generated_ids,
    skip_special_tokens=False,
)[0].lstrip()

# Populate document
doctags_doc = DocTagsDocument.from_doctags_and_image_pairs([doctags], [images[3]])
print(doctags)
# create a docling document
doc = DoclingDocument(name="Document")
doc.load_from_doctags(doctags_doc)

# export as any format
# HTML
# Path("Out/").mkdir(parents=True, exist_ok=True)
# output_path_html = Path("Out/") / "example.html"
# doc.save_as_html(output_path_html)
# MD
print(doc.export_to_markdown())


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<doctag><page_header><loc_14><loc_6><loc_71><loc_11>Table of Contents</page_header>
<section_header_level_1><loc_168><loc_26><loc_332><loc_31>AMAZON.COM, INC.</section_header_level_1>
<text><loc_185><loc_33><loc_315><loc_38>CONSOLIDATED STATEMENTS OF OPERATIONS</text>
<text><loc_190><loc_35><loc_314><loc_40>(in millions, except per share data)</text>
<text><loc_222><loc_41><loc_268><loc_46>(unaudited)</text>
<otsl><loc_14><loc_46><loc_490><loc_250><ecel><ched>Three Months Ended  March 31,<lcel><lcel><nl><ched>Net product sales<ched>2023<ched>2024<ched>2024<nl><rhed>Net service sales<fcel>$ 56,981<fcel>$ 60,915<fcel>$ 60,915<nl><rhed>Total net sales<fcel>70,377<fcel>82,398<fcel>127,358<nl><rhed>Operating expenses:<fcel>143,313<fcel>143,313<fcel>127,358<nl><rhed>Cost of sales<fcel>67,791<fcel>72,633<fcel>67,791<nl><rhed>Fulfillment<fcel>20,905<fcel>22,317<fcel>20,905<nl><rhed>Technology and infrastructure<fcel>20,450<fcel>20,424<fcel>20,450<nl><rhed>Sales and marketing<fcel>10,172<fcel>9